# Получение краткого содержания новости и ответов на вопросы с использованием предобученного BERT

В сегодняшнем воркшопе мы с вами познакомимся с DeepPavlov. DeepPavlov - это библиотека обученных deep learning моделей для самых разных задач NLP, разработанная в МФТИ. В том числе там есть и множество моделей, предобученных на русскоязычных датасетах, что особенно отрадно для русского исследователя.

Большинство моделей построены с использованием Tensorflow/Keras, но есть и модели для PyTorch.

В этом воркшопе мы суммаризируем текст и попробуем получить ответы на вопросы по нему с использованием архитектуры BERT. Также посмотрим на то, как можно использовать предобученные эмбеддинги из DeepPavlov.

Установим DeepPavlov. Обратите внимание, что в настоящее время поддерживаются версии Python 3.6 и 3.7. Также, если вы работаете под Windows, у вас должен быть установлен Git и Visual Studio 2015/2017 с компилятором для С++.

In [ ]:
!pip install deeppavlov

In [ ]:
У DeepPavlov немного нестандартная структура. Изначально вы не устанавливаете все модели, которые в нём имеются, ввиду их большого веса. После того, как вы установили фреймворк, вам нужно выбрать нужные модели, прописать для них конфигурации (можно использовать стандартные) и докачать их.

Давайте для начала импортируем модуль и посмотрим на имеющиеся модели.

In [1]:
import deeppavlov as dp

Список возможных моделей вместе с файлами конфигурации хранится в `dp.configs`.

In [2]:
dp.configs.keys()

frozenset({'classifiers',
           'doc_retrieval',
           'embedder',
           'entity_extraction',
           'faq',
           'kbqa',
           'morpho_syntax_parser',
           'multitask',
           'ner',
           'odqa',
           'ranking',
           'regressors',
           'relation_extraction',
           'russian_super_glue',
           'sentence_segmentation',
           'spelling_correction',
           'squad'})

Модели разделены по задачам, для решения которых они предназначены. Давайте посмотрим, какие модели имеются для решения задачи суммаризации текста.

In [3]:
dp.configs.summarization

AttributeError: 'Struct' object has no attribute 'summarization'

Имеется 2 модели, отличащиеся только косметически. Фактически это один и тот же BERT, предобученный на задачах Masked Language Modeling (MLM) и Next Sentence Prediction (NSP): он предсказывает, какое из предложений наиболее вероятно должно следовать после текущего. В `bert_as_summarizer_with_init` мы можем сами задать, какое предложение будет выступать в качестве первого. В случае обработки новостей, например, логичнее всего в этом качестве использовать заголовок новости. В `bert_as_summarizer` в качестве первого предложения выступает первое предложение исходного текста. 

Мы будем использовать `bert_as_summarizer_with_init`, подавая в качестве опорного предложения заголовок новости.

Но сперва давайте посмотрим, как выглядит конфигурационный файл для нашей модели.

In [4]:
with open(dp.configs.summarization.bert_as_summarizer_with_init, 'r') as file:
    print(file.read())

{
    "chainer": {
      "in": ["texts", "init_sentences"],
      "pipe": [
        {
          "class_name": "bert_as_summarizer",
          "bert_config_file": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_config.json",
          "pretrained_bert": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt",
          "vocab_file": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/vocab.txt",
          "max_summary_length": 100,
          "max_summary_length_in_tokens": true,
          "lang": "ru",
          "do_lower_case": false,
          "max_seq_length": 512,
          "in": ["texts", "init_sentences"],
          "out": ["summarized_text"]
        }
      ],
      "out": ["summarized_text"]
    },
    "metadata": {
      "variables": {
        "ROOT_PATH": "~/.deeppavlov",
        "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
        "MODELS_PATH": "{ROOT_PATH}/models",
        "CONFIGS_PATH": "{DEEPPAVLOV_PATH}/configs"
      },

В конфигурационном файле указываются пути для используемых моделей и словарей, описываются форматы входных и выходных данных а также указываются некоторые параметры задачи. Нас, например, интересуют `max_summary_length` и `max_summary_length_in_tokens`. В первом параметре указывается максимальная длинна саммари, которое должно получиться. Во втором - в чём она измеряется (в токенах или в предложениях).

Для нас не очень удобно измерять длину в токенах, тем более, что в BERT зачастую 1 токен не всегда равен одному слову (слова могут биться на несколько токенов). Поэтому давайте изменим этот конфиг так, чтобы длина составляла, например, 5 предложений.

Для этого скопируем файл и изменим соответствующие параметры.

In [5]:
# посмотрим изменённый файл

with open('bert_as_summarizer_with_init.json', 'r') as file:
    print(file.read())

{
    "chainer": {
      "in": ["texts", "init_sentences"],
      "pipe": [
        {
          "class_name": "bert_as_summarizer",
          "bert_config_file": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_config.json",
          "pretrained_bert": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/bert_model.ckpt",
          "vocab_file": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v2/vocab.txt",
          "max_summary_length": 5,
          "max_summary_length_in_tokens": false,
          "lang": "ru",
          "do_lower_case": false,
          "max_seq_length": 512,
          "in": ["texts", "init_sentences"],
          "out": ["summarized_text"]
        }
      ],
      "out": ["summarized_text"]
    },
    "metadata": {
      "variables": {
        "ROOT_PATH": "./deeppavlov",
        "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
        "MODELS_PATH": "{ROOT_PATH}/models",
        "CONFIGS_PATH": "{DEEPPAVLOV_PATH}/configs"
      },
 

Как уже говорилось выше, для использования модели её придётся скачать. Но предварительно придётся скачать и все необходимые зависимости, которые указаны в конфиге. 

Сделать это можно соответствующей командой:

In [ ]:
!python -m deeppavlov install bert_as_summarizer_with_init.json

Построить модель, предварительно её скачав, можно при помощи функции `build_model`. При первом выполнении придётся подождать скачивания из репозитория.

In [6]:
model = dp.build_model('bert_as_summarizer_with_init.json', download=True)

2020-11-17 03:41:50.530 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v2.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\G0nZaleZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\G0nZaleZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\G0nZaleZ\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\G0nZaleZ\AppData\Roaming\nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!









The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-11-17 03:42:00.773 INFO in 'deeppavlov.models.bert.bert_as_summarizer'['bert_as_summarizer'] at line 102: [initializing model with Bert from C:\Users\G0nZaleZ\PycharmProjects\ws-deeppavlov\deeppavlov\downloads\bert_models\rubert_cased_L-12_H-768_A-12_v2\bert_model.ckpt]


Модель загружена!

Давайте теперь познакомимся с текстом, с которым нам предстоит работать. Это новость о создании GPT-3: https://nplus1.ru/news/2020/05/29/gpt-3

In [7]:
title = "Третье поколение алгоритма OpenAI научилось выполнять текстовые задания по нескольким примерам."

text = """
Исследователи из OpenAI представили GPT-3 — алгоритм, который может выполнять разные задания по написанию текста на основе всего нескольких примеров. В новой версии используется та же архитектура, что и в предыдущем алгоритме GPT-2, однако разработчики увеличили количество используемых в модели параметров до 175 миллиардов, обучив модель на 570 гигабайтах текста. В итоге GPT-3 может отвечать на вопросы по прочитанному тексту, писать стихи, разгадывать анаграммы, решать простые арифметические примеры и даже переводить — и для этого ей нужно немного (от 10 до 100) примеров того, как именно это делать. Подробное описание работы алгоритма исследователи выложили на arXiv.org.

Важное ограничение современных алгоритмов NLP (natural language processing) — зависимость от контекста: многие алгоритмы могут выполнять только те задачи, на выполнение которых они обучены. Например, если необходим алгоритм, который пишет стихи, его нужно обучить на большом корпусе стихов — желательно, в том стиле, в котором должно быть итоговое. Если обучение пройдет успешно, алгоритм сможет произвести что-то похожее на стих, но вот ответить на вопрос или составить список слов для кроссворда он уже не сможет.

То, сколько данных понадобится для обучения NLP-алгоритма конкретной задаче, напрямую зависит от того, как алгоритм предобучен: если системе хорошо известны все требования грамматики языка, а генерировать осмысленные фразы он умеет изначально, то конкретно для обучения какой-то отдельной задаче нужно не так много данных. Задача, поэтому, сводится к тому, чтобы сделать предобученный NLP-алгоритм универсальным — таким, чтобы он фактически умел делать все, используя для обучения минимальное количество данных.

Для решения этой задачи команда исследователей из компании OpenAI под руководством Тома Брауна (Tom Brown) представила GPT-3. Этот NLP-алгоритм основан на предыдущей версии, представленной в феврале прошлого года: GPT-2, одна из самых используемых и продвинутых NLP-моделей, обучена на 40 гигабайтах текста, а ее метазадача заключается в том, чтобы предсказывать следующее слово в тексте. Как и ее предшественник GPT, GPT-2 основана на архитектуре Transformer.

Для обучения алгоритма исследователи собрали датасет из 570 гигабайтов текста, в который были включены данные проекта Common Crawl, вся Википедия, два датасета с книгами и вторая версия датасета WebText, содержащая тексты с веб-страниц (первую версию WebText использовали для обучения GPT-2). Исследователи обучили восемь разных моделей GPT-3: они отличались количеством параметров, которые модель устанавливала в ходе обучения (количество параметров, в свою очередь, зависело от количества слоев — при этом архитектура использовалась одна и та же). Внутри самой простой модели использовали 125 миллионов параметров, а в финальной GPT-3 — 175 миллиардов.

Задача, которую GPT-3 нужно было выполнить, заключалась в ответе на вопрос или в выполнении задания. Это могло было быть, например, «написать стихотворение», «разобрать анаграмму» или «прочитать текст и ответить на вопрос». Предобученной GPT-3 для выполнения задания (всего заданий было 42), помимо формулировки задания, давали либо один пример, либо несколько примеров (классически от 10 до 100 — столько, сколько модели будет необходимо, хотя в некоторых заданиях модели хватало и пяти примеров).

Несмотря на то, что точность каждого способа обучения модели возрастала с количеством определенных в модели параметров, обучение по нескольким примерам оказалось самым эффективным: по всем 42 заданиям точность при 175 миллиардах параметров составила почти 60 процентов. Например, при обучении на 64 примерах из датасета TriviaQA, который создан для обучения моделей понимать текст и отвечать на вопросы по прочитанному материалу, GPT-3 со 175 миллиардами параметров оказалась точна в 71,2 процента случаев — это чуть точнее, чем модель SOTA, которая обучена исключительно отвечать на вопросы по TriviaQA.

По нескольким примерам предобученная GPT-3 может писать тексты на заданную тему, придумывать стихи в определенном стиле, разгадывать анаграммы, решать простые арифметические примеры, отвечать на вопросы по прочитанному тексту. Кроме того, модель может переводить на несколько языков: при сборе данных ученые не ограничили язык текстов, поэтому семь процентов всего датасета — тексты на иностранных языках, которые также используются моделью для перевода по нескольким примерам.

Как и в случае с GPT-2, исследователи в препринте высказали свое беспокойство по поводу того, что разработанная ими модель может быть использована во вред — поэтому ее они пока что не предоставили. На странице разработчиков на GitHub можно найти кусок датасета и примеры заданий, которые использовались в работе.

В последние годы OpenAI преуспел не только в NLP-алгоритмах: в прошлом году разработчики компании представили алгоритмы, которые могут придумывать новую музыку и собирать кубик Рубика.
"""

Получаем саммари, передав текст и первое предложение в нашу модель. Как обычно, модель может обрабатывать данные батчами, но в нашем случае это будет батч из одного примера.

In [8]:
summary = model([text], [title])

In [9]:
# посмотрим результат

for sent in summary[0]:
    print(sent)

Третье поколение алгоритма OpenAI научилось выполнять текстовые задания по нескольким примерам.
Например, при обучении на 64 примерах из датасета TriviaQA, который создан для обучения моделей понимать текст и отвечать на вопросы по прочитанному материалу, GPT-3 со 175 миллиардами параметров оказалась точна в 71,2 процента случаев — это чуть точнее, чем модель SOTA, которая обучена исключительно отвечать на вопросы по TriviaQA.
Для обучения алгоритма исследователи собрали датасет из 570 гигабайтов текста, в который были включены данные проекта Common Crawl, вся Википедия, два датасета с книгами и вторая версия датасета WebText, содержащая тексты с веб-страниц (первую версию WebText использовали для обучения GPT-2).
В итоге GPT-3 может отвечать на вопросы по прочитанному тексту, писать стихи, разгадывать анаграммы, решать простые арифметические примеры и даже переводить — и для этого ей нужно немного (от 10 до 100) примеров того, как именно это делать.
Предобученной GPT-3 для выполнения 

Ещё одна из интересных задач NLP - ответы на вопросы по тексту (Question Answering). BERT тоже с этим успешно справляется, обучаясь ставить в тексте "отметки" в местах начала и конца ответа на вопрос.

В DeepPavlov есть несколько предобученных BERT-ов для этой задачи на русском языке. Установим стандартный (здесь мы уже ничего в конфигах править не будем, просто скопируем стандартный):

In [ ]:
!python -m deeppavlov install squad_ru.json

In [10]:
# скачаем модель для QA

model_qa = dp.build_model('squad_ru.json', download=True)

2020-11-17 03:46:33.280 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_1.4_cpu_compatible.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_1.4_cpu_compatible.tar.gz download because of matching hashes
2020-11-17 03:46:41.440 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec download because of matching hashes
2020-11-17 03:46:41.477 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize-char

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.



2020-11-17 03:46:55.317 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from C:\Users\G0nZaleZ\.deeppavlov\models\squad_model_ru\model]
INFO:deeppavlov.core.models.tf_model:[loading model from C:\Users\G0nZaleZ\.deeppavlov\models\squad_model_ru\model]


INFO:tensorflow:Restoring parameters from C:\Users\G0nZaleZ\.deeppavlov\models\squad_model_ru\model


INFO:tensorflow:Restoring parameters from C:\Users\G0nZaleZ\.deeppavlov\models\squad_model_ru\model


Попробуем найти в тексте ответы на следующие вопросы:

In [11]:
questions = ["Как называется алгоритм, занимающийся написанием текстов?",
             "Сколько текста было в обучающем датасете?",
             "Какое ограничение у современных алгоритмов NLP?"]

In [12]:
# найдём ответ для каждого вопроса

for q in questions:
    print(q)
    answer = model_qa([text], [q])[0][0]
    print(answer)

Как называется алгоритм, занимающийся написанием текстов?
GPT-3
Сколько текста было в обучающем датасете?
570 гигабайтов
Какое ограничение у современных алгоритмов NLP?
зависимость от контекста


Также в Deeppavlov есть предтренированные на больших датасетах эмбеддинги, которые вы можете использовать в своей работе подобно тому, как мы использовали однажды предварительно натренированный word2vec.

Помимо таких "тяжеловесов", как BERT, есть, например, и более простые модели. Например, ELMO - это эмбеддинг, построенный при помощи LSTM, который может назначать слову разные вектора в зависимости от контекста, в котором оно используется. Есть разные его варианты, обученные на разных датасетах. Например, на русской Википедии. Скачаем его (тоже воспользовавшись стандартным конфигом).

In [ ]:
# устанавливаем зависимости

!python -m deeppavlov install elmo_ru_wiki.json

In [13]:
# качаем и строим модель

elmo_emb = dp.build_model('elmo_ru_wiki.json', download=True)

2020-11-17 03:50:25.447 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz download because of matching hashes


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [14]:
# можно "закодировать" все необходимые слова

texts = ['Получаем эмбеддинг для слов этого предложения.',
         'Конечно, можно использовать несколько предложений. В качестве текста. Токенайзер справится.',
         'Слово']

embs = elmo_emb(texts)

In [15]:
# на выходе получаем список эмбеддингов
# т.к. все они разной длины (для разных предложений)
# они упаковываются в обычный список, а не np-массив

embs

[array([[-0.52713317,  0.04464398,  0.02200007, ...,  0.39243197,
         -0.07147413,  1.7890232 ],
        [-0.5409472 ,  0.07987181,  0.6760626 , ...,  0.22213186,
          0.30867967, -0.42369205],
        [ 0.15041974,  0.6983181 ,  0.25723952, ...,  0.10992923,
         -0.5024707 ,  0.24565211],
        ...,
        [ 0.19936937,  0.5070391 , -0.1287077 , ..., -0.08864522,
          0.3586684 ,  0.35104832],
        [-0.25707465,  0.67701054,  0.21267587, ...,  0.16995324,
         -0.8818088 ,  0.78075826],
        [-0.6920643 , -0.449081  , -1.3315636 , ..., -0.08664615,
         -0.13441339, -0.23009208]], dtype=float32),
 array([[-0.3380093 ,  0.38195705, -1.3128859 , ...,  0.92860615,
          0.35657376,  0.6304276 ],
        [ 0.18537113,  0.10103117, -0.75799304, ...,  0.31713006,
         -0.39594996,  0.10783564],
        [-0.8533196 , -0.69447803, -1.2852573 , ..., -0.14508472,
         -1.047672  ,  0.1478431 ],
        ...,
        [-0.42853767,  1.0058446 , -0.1

In [16]:
# посмотрим на размерность эмбеддингов

embs[0].shape

(7, 2560)

Данный эмбеддинг кодирует в том числе и знаки препинания. В частности, можно видеть, что все предложения заканчиваются одним вектором, соответствующим точке. 

Но в последнем предложении всего 1 вектор, т.к. оно состоит всего лишь из одного слова. 

In [17]:
embs[2].shape

(1, 2560)

Эмбеддинг очень объёмный: 2560 измерений. Однако такие сложные вектора могут помочь хорошо уловить контекст.

Обучить подобную модель в домашних условиях достаточно сложно, но зато вы можете пользоваться ей в своих разработках. Просто прогоняйте через неё текст перед тем, как отправить на обучение или обработку в свою модель для NLP-задачи, и это значительно улучшит качество вашей работы. 